## Zero Arb On chain Study 

This study compares on-chain settlement prices of COW protocol vs UNIv3. 

It filters all the trades on both sides that have the same block, same buy token and same sell token. 

In particular, it compares the cow price with 1)the best available Univ3 price in the block called `univ3BlockMin` and 2) the volume weighted average price of univ3 in the block, the `univ3VWAP` price. 

The best available price given by `univ3BlockMin` represents the best price univ3 executed at but does not take into consideration the volumes of these trades. A higher volume trade is expected to fetch a worse price given the impact to the poolso its not fair to compare the price of a small trade with a price of a high volume trade. To adjust for this, the vwap price of the block is constructed using the UNiv3 trades and allows one to compare the price obtained by cow to the volume weighted price of the block.  

This study only looks at Univ3 as proof of concept of methodology, but once peer reviewed, can be easily extended to include all major DEXs and aggregators that settle on chain, and extending the dataset. The hit rate of trades that match is roughly 2.7% This means that going over 5,000 COW trades, 136 trades happen to have corresponding UNIv3 trades of the same tokens in the same block.Adding more DEXs is expected to improve this, and would enrich the dataset. To our knowledge, no such study for COW protocol pricing comparison exists. We believe the COW ecosystem would benefit from an extention of this study to include other DEXs and fetching more laborious details such as UNI gas adjustments. 

The sources of data used are the COW and UNIv3 subgraph. Where richer data does exist under paid services, we wish to keep the study open source for anyone to run, play around with and verify. 


Notes on the study:

- If f a block has one univ3 trade, then `univ3BlockMin` = `univ3vwap` 
- COW prices retrieves already include all fees including gas prices. So to compare fairly, we adjust the UNIv3 price to reflect the gas fees as well. However, difficulty in retrieving exact data from open source data, meant that some heurestics were used. Details are explained in relevent cells. 


Outline: 

1. COW Trades : Obtain the last COW trades for a given query size.
2. UNI Swaps: Obtain UNI swaps that correspond to the same period as the COW dataset. 
3. Adjust UNI prices with gas fees. 
3. Merge COW with UNI on timestamp, token1, token2 : This results in the trades that are overlapping only. 
4. Group UNI trades together and derive one price per block. Once deriving the one price to be the Block Max price and second the VWAP. 
5. Derive stats

### 1. COW Trades

In [2]:
from datastreams.datastream import Streamer

import matplotlib.pyplot as plt
import pandas as pd
import polars as pl

# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [2]:
# instantiate Streamer class. Note that we need two separate streamer classes, otherwise the queries will be overwritten. 
cow_ds = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')
#cow_ds2 = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')

In [3]:
# DEFINE TIMESTAMP HERE. Timstamp is used for replication quality assurance purposes. 
timestamp =  1646092800

# we set a fixed query size number. The Cow settlements and Uniswap swaps query are multiples larger than this initial query size.
query_size = 500000

#Filter size - We filter trades out that are smaller than $1000 USD size
filter_usd = 100

In [5]:
# query COW schema: trades
trades_fp = cow_ds.queryDict.get('trades')

# trades query path that gets token a -> token b trades
trades_qp = trades_fp(
    first=query_size,
    orderBy='timestamp',
    orderDirection='desc',
    where = {
    'timestamp_gt': timestamp,
    'buyAmountUsd_gt': filter_usd
    }
)

# run query
trades_df = cow_ds.runQuery(trades_qp)

FIELD - trades


In [7]:
print(f'query returned {len(trades_df)} rows')

query returned 267367 rows


In [8]:
# get unique values in trades_df column to verify the query results.
len(trades_df['trades_buyToken_id'].unique())

470

In [9]:
trades_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         object
trades_txHash            object
trades_settlement_id     object
trades_buyAmount         object
trades_sellAmount        object
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
trades_buyAmountEth     float64
trades_sellAmountEth    float64
trades_buyAmountUsd     float64
trades_sellAmountUsd    float64
endpoint                 object
dtype: object

In [10]:
trades_df.head(20)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_buyAmount,trades_sellAmount,trades_sellToken_id,trades_buyToken_id,trades_order_id,trades_buyAmountEth,trades_sellAmountEth,trades_buyAmountUsd,trades_sellAmountUsd,endpoint
0,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b|0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af|126,1679120339,15532267543,3165845034591732,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,4172524128209352386385,8000000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b,7.946574,8.000000,1.441464e+04,1.451155e+04,cow
1,0xf06e7497bb98d4aa16a3ae307a1536b96ff97c6a0cba896b2eec2a202cafa8e4cb954f2a0973abc92f9f8ed58938821b219cfe8064155e7c|0x4b84cf411af0d732e991569cc2a61816402173cf3c4c1efaa96ae6e8cfeaab0c|101,1679120303,14252233437,14379699810404239360,0x4b84cf411af0d732e991569cc2a61816402173cf3c4c1efaa96ae6e8cfeaab0c,0x4b84cf411af0d732e991569cc2a61816402173cf3c4c1efaa96ae6e8cfeaab0c,29225677076,25146192644758883166303,0x0f2d719407fdbeff09d87557abb7232601fd9f29,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xf06e7497bb98d4aa16a3ae307a1536b96ff97c6a0cba896b2eec2a202cafa8e4cb954f2a0973abc92f9f8ed58938821b219cfe8064155e7c,16.111677,16.315002,2.922568e+04,2.959450e+04,cow
2,0x6fb7c98f5e1c759337f9f349f1456d9a7294d9aacaff2ec07e62cb36ef4842490775ea2a88db33038c4069e41dda8d963b64c2a864155e12|0x98005cb4731f20a8bce06783c6f94157392f1dedf327d0a794f1eb5b0b288952|87,1679120195,15356091286,3650773,0x98005cb4731f20a8bce06783c6f94157392f1dedf327d0a794f1eb5b0b288952,0x98005cb4731f20a8bce06783c6f94157392f1dedf327d0a794f1eb5b0b288952,7415169299093985603,13424652237,0xdac17f958d2ee523a2206206994597c13d831ec7,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x6fb7c98f5e1c759337f9f349f1456d9a7294d9aacaff2ec07e62cb36ef4842490775ea2a88db33038c4069e41dda8d963b64c2a864155e12,7.415169,7.400809,1.345070e+04,1.342465e+04,cow
3,0xe247d6589242d786c1e403f96cbc81eb68f5a6e5d4e5c00cae893b2bdb1358dde495d8c21bb0c4ab9a627627a4016df40c6daa7464155d9a|0x08cbed0d0d293c975f25165b568cf38b95eda9136bb0c2e947b62a1ddf5d1df4|88,1679120063,15295085721,7341347,0x08cbed0d0d293c975f25165b568cf38b95eda9136bb0c2e947b62a1ddf5d1df4,0x08cbed0d0d293c975f25165b568cf38b95eda9136bb0c2e947b62a1ddf5d1df4,31840511,8791442860,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xe247d6589242d786c1e403f96cbc81eb68f5a6e5d4e5c00cae893b2bdb1358dde495d8c21bb0c4ab9a627627a4016df40c6daa7464155d9a,4.834482,4.846590,8.769480e+03,8.791443e+03,cow
4,0x79e314443f73b835dd68221d6028458de7c31f840a4a2419e1d93f06d9d479952d4c76b73c0aeb8faf27acfdb242062eaca1a3bd64155c32|0x6c0eac8ba0f5b8ac04aab4f7446ae3e3bb2b5330e324e413bbd682560c7ab402|139,1679119715,13796611695,1263938114594753,0x6c0eac8ba0f5b8ac04aab4f7446ae3e3bb2b5330e324e413bbd682560c7ab402,0x6c0eac8ba0f5b8ac04aab4f7446ae3e3bb2b5330e324e413bbd682560c7ab402,100000000000000000,91180590273088153,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x79e314443f73b835dd68221d6028458de7c31f840a4a2419e1d93f06d9d479952d4c76b73c0aeb8faf27acfdb242062eaca1a3bd64155c32,0.100000,0.101456,1.813944e+02,1.840362e+02,cow
5,0x028ae7288cce44c1e2259fa9628d42b969659d3ec1422fff1998c6195fe271c50c6ac3fcea667fd6c62483ce1dbbce6f6ce0fb1f64156bb1|0xc50ea5c8d76d9b6b30d5e8b6ba2d1da34440086237f3972fcef63d24d0434b63|204,1679119475,16952004893,462399620936909824,0xc50ea5c8d76d9b6b30d5e8b6ba2d1da34440086237f3972fcef63d24d0434b63,0xc50ea5c8d76d9b6b30d5e8b6ba2d1da34440086237f3972fcef63d24d0434b63,157415751171139074,50000000000000000000,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x028ae7288cce44c1e2259f

In [11]:
# Set query size large enough to retrieve all tokens in the graph
query_size_token = 10000000

token_fp = cow_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
token_qp = token_fp(
    first=query_size_token,
)

# run query
token_df = cow_ds.runQuery(token_qp)

FIELD - tokens


In [12]:
# Merge1 on sell token address.  x = sell token 

merged_df = pd.merge(trades_df, token_df, left_on='trades_sellToken_id', right_on='tokens_address')

In [13]:
# Merge2 on buy token address.  y = buy token 
merged_df2 = pd.merge(merged_df, token_df, left_on='trades_buyToken_id', right_on='tokens_address')

In [14]:
merged_df2 = merged_df2.rename(columns={
    "tokens_symbol_x": "sell_token_symbol",
    "tokens_symbol_y":"buy_token_symbol",
    "tokens_decimals_x": "sell_token_decimal", 
    "tokens_decimals_y": "buy_token_decimal"})

In [15]:
# filter necessary columns
cow_df = merged_df2[[
    'trades_id',
    'trades_timestamp', 
    'trades_gasPrice', 
    'trades_feeAmount',                 
    'trades_txHash',                    
    'trades_settlement_id',   
    'trades_sellAmount',
    'sell_token_decimal',
    'trades_buyAmount',   
    'buy_token_decimal',
    'trades_sellToken_id',              
    'trades_buyToken_id',               
    'trades_order_id',                  
    'sell_token_symbol',
    'buy_token_symbol',
    'trades_sellAmountUsd'
    ]]


In [16]:
cow_df.loc[:, 'sell_amount'] = cow_df['trades_sellAmount'] / (10**cow_df['sell_token_decimal'])
cow_df.loc[:, 'buy_amount'] = cow_df['trades_buyAmount'] / (10**cow_df['buy_token_decimal'])


/tmp/ipykernel_109594/1103516907.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df.loc[:, 'sell_amount'] = cow_df['trades_sellAmount'] / (10**cow_df['sell_token_decimal'])
/tmp/ipykernel_109594/1103516907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df.loc[:, 'buy_amount'] = cow_df['trades_buyAmount'] / (10**cow_df['buy_token_decimal'])


In [17]:
# calculate COW price defined sell amount / buy amount
cow_df['cow_sell_buy_ratio'] = cow_df['sell_amount'] / cow_df['buy_amount']

/tmp/ipykernel_109594/3636551783.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df['cow_sell_buy_ratio'] = cow_df['sell_amount'] / cow_df['buy_amount']


In [18]:
cow_df.shape

(267367, 19)

In [19]:
cow_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         object
trades_txHash            object
trades_settlement_id     object
trades_sellAmount        object
sell_token_decimal        int64
trades_buyAmount         object
buy_token_decimal         int64
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
sell_token_symbol        object
buy_token_symbol         object
trades_sellAmountUsd    float64
sell_amount              object
buy_amount               object
cow_sell_buy_ratio       object
dtype: object

In [20]:
cow_df.head(5)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,trades_sellAmountUsd,sell_amount,buy_amount,cow_sell_buy_ratio
0,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b|0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af|126,1679120339,15532267543,3165845034591732,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,8000000000000000000,18,4172524128209352386385,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b,WETH,AURA,14511.550918,8.0,4172.524128,0.001917
1,0x3324082f260815659b5c7b1af06f50b49e216fdcc1c21b9b49d0fc4804b80c59892dae4ba73a7984dff8d888c93a119a4cf2c89264143c2c|0x3bab8a8ba9673c9e70be15ace75ea82539105ce1434582bb9d1d4cf7424365f0|62,1679041787,17225846628,1727119053718436,0x3bab8a8ba9673c9e70be15ace75ea82539105ce1434582bb9d1d4cf7424365f0,0x3bab8a8ba9673c9e70be15ace75ea82539105ce1434582bb9d1d4cf7424365f0,948928969753779530,18,466742577410935985899,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x3324082f260815659b5c7b1af06f50b49e216fdcc1c21b9b49d0fc4804b80c59892dae4ba73a7984dff8d888c93a119a4cf2c89264143c2c,WETH,AURA,1633.374139,0.948929,466.742577,0.002033
2,0x6a789dcbc8847ee6a87d10c59196378194c7db47d966f32312a55d76ffcf9e4d40a50cf069e992aa4536211b23f286ef88752187ffffffff|0xb5daebce5552a63a8c1c3245bcd7402bf93dffa6de5dc0e55b4f41eb9194c896|106,1678909319,25488256122,3775644729595806,0xb5daebce5552a63a8c1c3245bcd7402bf93dffa6de5dc0e55b4f41eb9194c896,0xb5daebce5552a63a8c1c3245bcd7402bf93dffa6de5dc0e55b4f41eb9194c896,4250000000000000000,18,2062411812585896630478,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x6a789dcbc8847ee6a87d10c59196378194c7db47d966f32312a55d76ffcf9e4d40a50cf069e992aa4536211b23f286ef88752187ffffffff,WETH,AURA,7020.909085,4.25,2062.411813,0.002061
3,0x7fabb5ac0ed4a556c5129b74724e6b6a92def8766ec4062b40495fd205909291c5dd3b6808b9025d824d835f59cc30b33e387f71641041d1|0xfa83e643bef1adefcf463e911e2fbc3b69e6748496df15c1dd3a2cc489ec1ef2|159,1678781075,17472664139,1723472527575756,0xfa83e643bef1adefcf463e911e2fbc3b69e6748496df15c1dd3a2cc489ec1ef2,0xfa83e643bef1adefcf463e911e2fbc3b69e6748496df15c1dd3a2cc489ec1ef2,104907418849112155,18,46947014136430612403,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x7fabb5ac0ed4a556c5129b74724e6b6a92def8766ec4062b40495fd205909291c5dd3b6808b9025d824d835f59cc30b33e387f71641041d1,WETH,AURA,176.037627,0.104907,46.947014,0.002235
4,0x08c09e7a85d81093304d83647fb890f20e40331abe31d5e24a4658deecf64bbfe125ea8fe5668ebd015c1a73b3ca359fb32cd20d64101584|0x0ed133abddca7c4ca9e1eff728dd1b2a9dceb95b808068990ace05296c3a1efc|37,1678774811,15020788199,0,0x0ed133abddca7c4ca9e1eff728dd1b2a9dceb95b808068990ace05296c3a1efc,0x0ed133abddca7c4ca9e1eff728dd1b2a9dceb95b808068990ace05296c3a1efc,89550075380465136,18,41040457639102084480,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x08c09e7a85d81093304d83647fb890f20e40331abe31d5e24a4658deecf64bbfe125ea8fe5668ebd015c1a73b3ca359fb32cd20d64101584,WETH,AURA,150.603185,0.08955,41.040458,0.002182


In [3]:
cow_df = pd.read_csv('cow_df.csv')

/tmp/ipykernel_237871/911048642.py:1: DtypeWarning: Columns (9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  cow_df = pd.read_csv('cow_df.csv')


In [4]:
# check first and last timestamps

first_timestamp_cow = cow_df['trades_timestamp'].min()
last_timestamp_cow = cow_df['trades_timestamp'].max()

print("First timestamp:", first_timestamp_cow)
print("Last timestamp:", last_timestamp_cow)

First timestamp: 1646092818
Last timestamp: 1679120339


In [5]:
import time

# Convert Unix timestamps to human-readable dates
first_date_cow = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(first_timestamp_cow))
last_date_cow = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(last_timestamp_cow))

print("First date:", first_date_cow)
print("Last date:", last_date_cow)

First date: 2022-03-01 00:00:18
Last date: 2023-03-18 06:18:59


In [6]:
timestamps_list = cow_df['trades_timestamp'].to_list()
# get unique values in cow_timestamps
cow_timestamps = list(set(timestamps_list))

In [7]:
len(cow_timestamps)

212300

### 2. Uniswap V3 Swaps

In [15]:
# instantiate Streamer object. |
# Note - unlike the cow queries, univ3 does not require multiple streamer instantations because the swaps field path is reset each iteration. 
# If the Cow queries were updated to use the same method, we could use the same streamer object for all queries.
univ3_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

In [16]:
# get a query field path from the query dictionary which is automatically populated in the Streamer object
uni_swaps_fp = univ3_ds.queryDict.get('swaps')

In [17]:
swaps_query_index = len(cow_timestamps)
swaps_query_index

212300

In [18]:
cow_timestamp_query_list = [i for i in range(0, len(cow_timestamps), 999)]
# append the last value in the index to the query_index_list
cow_timestamp_query_list.append(swaps_query_index)

In [19]:
swaps_data = []

In [25]:
for i in range(1, len(cow_timestamp_query_list)):
    print(f'querying {cow_timestamp_query_list[i-1]} to {cow_timestamp_query_list[i]}. Remaining: {len(cow_timestamps) - cow_timestamp_query_list[i]}, {((len(cow_timestamps) - cow_timestamp_query_list[i]) / len(cow_timestamps)) * 100:.2f}%')
    
    # define a "partition", which is part of the larger cow_timestamps
    partition = cow_timestamps[cow_timestamp_query_list[i-1]:cow_timestamp_query_list[i]]

    # define the query path of unique settelement ids
    uni_swaps_qp = uni_swaps_fp(
        orderBy=uni_swaps_fp.timestamp,
        orderDirection='desc',
        first=100000,
        where={'timestamp_in': partition, 
               'amountInUSD_gt': 100,
               'amountOutUSD_gt': 100}
    )

    # run uni swaps query
    df = univ3_ds.runQuery(uni_swaps_qp)
    
    # convert all int64 columns to float64 to avoid large int overflows
    df['swaps_gasLimit'] = df['swaps_gasLimit'].astype('float64')
    df['swaps_gasPrice'] = df['swaps_gasPrice'].astype('float64')
    df['swaps_tick'] = df['swaps_tick'].astype('float64')
    df['swaps_amountIn'] = df['swaps_amountIn'].astype('float64')
    df['swaps_amountOut'] = df['swaps_amountOut'].astype('float64')
    

    if df.empty:
        print('uni swaps empty')
        continue
    else:
        swaps_data.append(df)
        

querying 0 to 999. Remaining: 211301, 99.53%
FIELD - swaps
querying 999 to 1998. Remaining: 210302, 99.06%
FIELD - swaps
querying 1998 to 2997. Remaining: 209303, 98.59%
FIELD - swaps
querying 2997 to 3996. Remaining: 208304, 98.12%
FIELD - swaps
querying 3996 to 4995. Remaining: 207305, 97.65%
FIELD - swaps
querying 4995 to 5994. Remaining: 206306, 97.18%
FIELD - swaps
querying 5994 to 6993. Remaining: 205307, 96.71%
FIELD - swaps
querying 6993 to 7992. Remaining: 204308, 96.24%
FIELD - swaps
querying 7992 to 8991. Remaining: 203309, 95.76%
FIELD - swaps
querying 8991 to 9990. Remaining: 202310, 95.29%
FIELD - swaps
querying 9990 to 10989. Remaining: 201311, 94.82%
FIELD - swaps
querying 10989 to 11988. Remaining: 200312, 94.35%
FIELD - swaps
querying 11988 to 12987. Remaining: 199313, 93.88%
FIELD - swaps
querying 12987 to 13986. Remaining: 198314, 93.41%
FIELD - swaps
querying 13986 to 14985. Remaining: 197315, 92.94%
FIELD - swaps
querying 14985 to 15984. Remaining: 196316, 92.47%


querying 125874 to 126873. Remaining: 85427, 40.24%
FIELD - swaps
querying 126873 to 127872. Remaining: 84428, 39.77%
FIELD - swaps
querying 127872 to 128871. Remaining: 83429, 39.30%
FIELD - swaps
querying 128871 to 129870. Remaining: 82430, 38.83%
FIELD - swaps
querying 129870 to 130869. Remaining: 81431, 38.36%
FIELD - swaps
querying 130869 to 131868. Remaining: 80432, 37.89%
FIELD - swaps
querying 131868 to 132867. Remaining: 79433, 37.42%
FIELD - swaps
querying 132867 to 133866. Remaining: 78434, 36.94%
FIELD - swaps
querying 133866 to 134865. Remaining: 77435, 36.47%
FIELD - swaps
querying 134865 to 135864. Remaining: 76436, 36.00%
FIELD - swaps
querying 135864 to 136863. Remaining: 75437, 35.53%
FIELD - swaps
querying 136863 to 137862. Remaining: 74438, 35.06%
FIELD - swaps
querying 137862 to 138861. Remaining: 73439, 34.59%
FIELD - swaps
querying 138861 to 139860. Remaining: 72440, 34.12%
FIELD - swaps
querying 139860 to 140859. Remaining: 71441, 33.65%
FIELD - swaps
querying 1

In [27]:
uni_swaps_df = pd.concat(swaps_data)

In [28]:
uni_swaps_df.shape

(1293378, 20)

In [29]:
uni_swaps_df = uni_swaps_df.drop_duplicates(subset=['swaps_id'])

In [30]:
uni_swaps_df.shape

(1293378, 20)

In [31]:
uni_swaps_df.dtypes

swaps_id               object
swaps_hash             object
swaps_nonce             int64
swaps_logIndex          int64
swaps_gasLimit        float64
swaps_gasUsed          object
swaps_gasPrice        float64
swaps_protocol_id      object
swaps_account_id       object
swaps_pool_id          object
swaps_blockNumber       int64
swaps_timestamp         int64
swaps_tick            float64
swaps_tokenIn_id       object
swaps_amountIn        float64
swaps_amountInUSD     float64
swaps_tokenOut_id      object
swaps_amountOut       float64
swaps_amountOutUSD    float64
endpoint               object
dtype: object

In [32]:
uni_swaps_df.head(5)

,swaps_id,swaps_hash,swaps_nonce,swaps_logIndex,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_protocol_id,swaps_account_id,swaps_pool_id,swaps_blockNumber,swaps_timestamp,swaps_tick,swaps_tokenIn_id,swaps_amountIn,swaps_amountInUSD,swaps_tokenOut_id,swaps_amountOut,swaps_amountOutUSD,endpoint
0,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b0d000000,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b,4997,13,120464.0,None,1.601539e+11,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x02c090592a12829ca350286391653efbc3d7c0c8,0x60594a405d53811d3bc4766596efd80fd545a270,16824178,1678772495,-74266.0,0x6b175474e89094c44da98b954eedeac495271d0f,2.822205e+23,282220.488073,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.679753e+20,282466.989679,uniswap-v3-ethereum
1,0x54cf49588c5e02f03e05f2a5506d76abef010def35e8a309b9167a8a4cc2ba5420000000,0x54cf49588c5e02f03e05f2a5506d76abef010def35e8a309b9167a8a4cc2ba54,592,32,1093736.0,None,1.653092e+10,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x004697d208819b5670dada8cc18c2dfb260bdd74,0x3778f73e949c98817a1f96c58121323754b429da,16824178,1678772495,71743.0,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,3.891817e+20,2384.497229,0x3b94440c8c4f69d5c9f47bab9c5a93064df460f5,5.113547e+23,2377.216475,uniswap-v3-ethereum
2,0x54cf49588c5e02f03e05f2a5506d76abef010def35e8a309b9167a8a4cc2ba541f000000,0x54cf49588c5e02f03e05f2a5506d76abef010def35e8a309b9167a8a4cc2ba54,592,31,1093736.0,None,1.653092e+10,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x004697d208819b5670dada8cc18c2dfb260bdd74,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,16824178,1678772495,-56150.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.422340e+18,2391.804387,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,3.891817e+20,2384.497467,uniswap-v3-ethereum
3,0x3130798c79fe3ac73cf544aa56922fab9e6a0332c6bade2af1b25be1d1407fbb03000000,0x3130798c79fe3ac73cf544aa56922fab9e6a0332c6bade2af1b25be1d1407fbb,5962,3,505954.0,None,1.709092e+10,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x2d8719cf0fe19782140ae16eec03938a96f5228b,0x60594a405d53811d3bc4766596efd80fd545a270,16824178,1678772495,-74265.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,4.250000e+19,71467.927559,0x6b175474e89094c44da98b954eedeac495271d0f,7.133300e+22,71332.997670,uniswap-v3-ethereum
4,0xfa8c7a1a6ae3b6d0ea7aff6e145447566f71ef9fd81c42af1a698d8cd4c31bd941010000,0xfa8c7a1a6ae3b6d0ea7aff6e145447566f71ef9fd81c42af1a698d8cd4c31bd9,4696,321,577346.0,None,1.732318e+10,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xb74e5e06f50fa9e4ef645efdad9d996d33cc2d9d,0xe0554a476a092703abdb3ef35c80e0d76d32939f,16824172,1678772423,202100.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.600000e+18,2690.557273,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2.679206e+09,2679.205750,uniswap-v3-ethereum


In [33]:
# check first and last timestamps

first_timestamp = uni_swaps_df['swaps_timestamp'].min()
last_timestamp = uni_swaps_df['swaps_timestamp'].max()

print("First timestamp:", first_timestamp)
print("Last timestamp:", last_timestamp)

First timestamp: 1646092818
Last timestamp: 1679120339


In [34]:
import time

# Convert Unix timestamps to human-readable dates
first_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(first_timestamp))
last_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(last_timestamp))

print("First date:", first_date)
print("Last date:", last_date)


First date: 2022-03-01 00:00:18
Last date: 2023-03-18 06:18:59


In [36]:
# Query uni tokens subgraph to retrieve token symbols and decimals

# Set query size large enough to retrieve all tokens in the graph
query_size_token = 10000

uni_token_fp = univ3_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
uni_token_qp = uni_token_fp(
    first=query_size_token,
)

# run query
uni_token_df = univ3_ds.runQuery(uni_token_qp)

FIELD - tokens


In [37]:
uni_token_df.shape

(7268, 7)

In [38]:
uni_token_df.dtypes

tokens_id                       object
tokens_name                     object
tokens_symbol                   object
tokens_decimals                  int64
tokens_lastPriceUSD            float64
tokens_lastPriceBlockNumber      int64
endpoint                        object
dtype: object

In [39]:
# Merge1 on tokenIN address.   x is in 

merge1_uni_df = pd.merge(uni_swaps_df, uni_token_df, left_on='swaps_tokenIn_id', right_on='tokens_id')

In [40]:
# Merge2 on tokenOUT address.  y is out 

merge2_uni_df = pd.merge(merge1_uni_df, uni_token_df, left_on='swaps_tokenOut_id', right_on='tokens_id')

In [41]:
merge2_uni_df.dtypes

swaps_id                          object
swaps_hash                        object
swaps_nonce                        int64
swaps_logIndex                     int64
swaps_gasLimit                   float64
swaps_gasUsed                     object
swaps_gasPrice                   float64
swaps_protocol_id                 object
swaps_account_id                  object
swaps_pool_id                     object
swaps_blockNumber                  int64
swaps_timestamp                    int64
swaps_tick                       float64
swaps_tokenIn_id                  object
swaps_amountIn                   float64
swaps_amountInUSD                float64
swaps_tokenOut_id                 object
swaps_amountOut                  float64
swaps_amountOutUSD               float64
endpoint_x                        object
tokens_id_x                       object
tokens_name_x                     object
tokens_symbol_x                   object
tokens_decimals_x                  int64
tokens_lastPrice

In [42]:
merge2_uni_df = merge2_uni_df.rename(columns={
    "tokens_symbol_x": "tokenIn_symbol",
    "tokens_symbol_y":"tokenOut_symbol",
    "tokens_decimals_x": "tokenIn_decimal", 
    "tokens_decimals_y": "tokenOut_decimal"})

In [43]:
# filter necessary columns
uni_df = merge2_uni_df[[
    'swaps_id',
    'swaps_hash',
    'swaps_protocol_id',
    'swaps_blockNumber',
    'swaps_timestamp',
    'swaps_tokenIn_id',
    'tokenIn_symbol',
    'tokenIn_decimal',
    'swaps_amountIn',
    'swaps_tokenOut_id',
    'tokenOut_symbol',
    'tokenOut_decimal',
    'swaps_amountOut',
    'swaps_gasLimit',                   
    'swaps_gasUsed',                     
    'swaps_gasPrice',                   
    'swaps_pool_id',
    'swaps_amountOutUSD' 
    ]]

In [44]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD
0,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b0d000000,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824178,1678772495,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,2.822205e+23,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.679753e+20,120464.0,None,1.601539e+11,0x60594a405d53811d3bc4766596efd80fd545a270,282466.989679
1,0x966bcc3b8e3dd0e13208208b2e95eafddf3c87813006fd36651dd342a4352acc16000000,0x966bcc3b8e3dd0e13208208b2e95eafddf3c87813006fd36651dd342a4352acc,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824149,1678772147,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,1.265266e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,7.520070e+18,120103.0,None,2.556527e+10,0x60594a405d53811d3bc4766596efd80fd545a270,12645.737410
2,0x386d24d52ebb99cb41a14bc76459ffcb9d3268ee81cebe8112325ecb5f19fed517000000,0x386d24d52ebb99cb41a14bc76459ffcb9d3268ee81cebe8112325ecb5f19fed5,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824099,1678771547,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,6.180051e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,3.676467e+19,153329.0,None,2.177077e+10,0x60594a405d53811d3bc4766596efd80fd545a270,61930.434502
3,0x3a948990f8d467535e0804c4108913f24d4394d39b5a325d135998f936146afd02000000,0x3a948990f8d467535e0804c4108913f24d4394d39b5a325d135998f936146afd,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824091,1678771451,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,8.504112e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,5.059155e+19,109912.0,None,3.209627e+11,0x60594a405d53811d3bc4766596efd80fd545a270,85221.960542
4,0xa13aca138e3cf246d46a449ab28acd2206ebf878459e68fe58e90de3f70b1edb72000000,0xa13aca138e3cf246d46a449ab28acd2206ebf878459e68fe58e90de3f70b1edb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824042,1678770863,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,4.022544e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2.365715e+19,1000000.0,None,1.166374e+11,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,40125.204944


In [45]:
# Get amounts in a uniform decimal 
uni_df.loc[:, 'univ3_In_amount'] = uni_df['swaps_amountIn'] / (10**uni_df['tokenIn_decimal'])
uni_df.loc[:, 'univ3_Out_amount'] = uni_df['swaps_amountOut'] / (10**uni_df['tokenOut_decimal'])

/tmp/ipykernel_237871/2168625794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df.loc[:, 'univ3_In_amount'] = uni_df['swaps_amountIn'] / (10**uni_df['tokenIn_decimal'])
/tmp/ipykernel_237871/2168625794.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df.loc[:, 'univ3_Out_amount'] = uni_df['swaps_amountOut'] / (10**uni_df['tokenOut_decimal'])


In [46]:
# calculate UNI swap price defined In amount / Out amount
uni_df['uni_in_out_ratio'] = uni_df['univ3_In_amount'] / uni_df['univ3_Out_amount']

/tmp/ipykernel_237871/1576826904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['uni_in_out_ratio'] = uni_df['univ3_In_amount'] / uni_df['univ3_Out_amount']


In [47]:
uni_df.shape

(1293378, 21)

In [48]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio
0,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b0d000000,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824178,1678772495,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,2.822205e+23,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.679753e+20,120464.0,None,1.601539e+11,0x60594a405d53811d3bc4766596efd80fd545a270,282466.989679,282220.488073,167.975307,1680.130808
1,0x966bcc3b8e3dd0e13208208b2e95eafddf3c87813006fd36651dd342a4352acc16000000,0x966bcc3b8e3dd0e13208208b2e95eafddf3c87813006fd36651dd342a4352acc,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824149,1678772147,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,1.265266e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,7.520070e+18,120103.0,None,2.556527e+10,0x60594a405d53811d3bc4766596efd80fd545a270,12645.737410,12652.662316,7.520070,1682.519152
2,0x386d24d52ebb99cb41a14bc76459ffcb9d3268ee81cebe8112325ecb5f19fed517000000,0x386d24d52ebb99cb41a14bc76459ffcb9d3268ee81cebe8112325ecb5f19fed5,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824099,1678771547,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,6.180051e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,3.676467e+19,153329.0,None,2.177077e+10,0x60594a405d53811d3bc4766596efd80fd545a270,61930.434502,61800.508369,36.764666,1680.975672
3,0x3a948990f8d467535e0804c4108913f24d4394d39b5a325d135998f936146afd02000000,0x3a948990f8d467535e0804c4108913f24d4394d39b5a325d135998f936146afd,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824091,1678771451,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,8.504112e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,5.059155e+19,109912.0,None,3.209627e+11,0x60594a405d53811d3bc4766596efd80fd545a270,85221.960542,85041.116105,50.591553,1680.935069
4,0xa13aca138e3cf246d46a449ab28acd2206ebf878459e68fe58e90de3f70b1edb72000000,0xa13aca138e3cf246d46a449ab28acd2206ebf878459e68fe58e90de3f70b1edb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824042,1678770863,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,4.022544e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2.365715e+19,1000000.0,None,1.166374e+11,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,40125.204944,40225.441874,23.657149,1700.350380


### 3. Adjust uni price by gas price

In [49]:
# convert gas prices to ETH. 
uni_df['gas_wei'] = uni_df['swaps_gasPrice']/10**18 

/tmp/ipykernel_237871/4059436116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['gas_wei'] = uni_df['swaps_gasPrice']/10**18


In [50]:
# Calculate gas in USD terms. 1500 price of ETH used as median. Better data source needed to get for a given timestamp. 
uni_df['gasUSD'] = uni_df['gas_wei'] * uni_df['swaps_gasLimit'] *1500 

/tmp/ipykernel_237871/4630247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['gasUSD'] = uni_df['gas_wei'] * uni_df['swaps_gasLimit'] *1500


In [51]:
uni_df.shape

(1293378, 23)

In [52]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD
0,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b0d000000,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824178,1678772495,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,2.822205e+23,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.679753e+20,120464.0,None,1.601539e+11,0x60594a405d53811d3bc4766596efd80fd545a270,282466.989679,282220.488073,167.975307,1680.130808,1.601539e-07,28.939167
1,0x966bcc3b8e3dd0e13208208b2e95eafddf3c87813006fd36651dd342a4352acc16000000,0x966bcc3b8e3dd0e13208208b2e95eafddf3c87813006fd36651dd342a4352acc,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824149,1678772147,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,1.265266e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,7.520070e+18,120103.0,None,2.556527e+10,0x60594a405d53811d3bc4766596efd80fd545a270,12645.737410,12652.662316,7.520070,1682.519152,2.556527e-08,4.605699
2,0x386d24d52ebb99cb41a14bc76459ffcb9d3268ee81cebe8112325ecb5f19fed517000000,0x386d24d52ebb99cb41a14bc76459ffcb9d3268ee81cebe8112325ecb5f19fed5,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824099,1678771547,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,6.180051e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,3.676467e+19,153329.0,None,2.177077e+10,0x60594a405d53811d3bc4766596efd80fd545a270,61930.434502,61800.508369,36.764666,1680.975672,2.177077e-08,5.007136
3,0x3a948990f8d467535e0804c4108913f24d4394d39b5a325d135998f936146afd02000000,0x3a948990f8d467535e0804c4108913f24d4394d39b5a325d135998f936146afd,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824091,1678771451,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,8.504112e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,5.059155e+19,109912.0,None,3.209627e+11,0x60594a405d53811d3bc4766596efd80fd545a270,85221.960542,85041.116105,50.591553,1680.935069,3.209627e-07,52.916485
4,0xa13aca138e3cf246d46a449ab28acd2206ebf878459e68fe58e90de3f70b1edb72000000,0xa13aca138e3cf246d46a449ab28acd2206ebf878459e68fe58e90de3f70b1edb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824042,1678770863,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,4.022544e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2.365715e+19,1000000.0,None,1.166374e+11,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,40125.204944,40225.441874,23.657149,1700.350380,1.166374e-07,174.956143


In [53]:
# Use the amountOutUSD as estimator of the trade volume and calculate the percentage of gas fees of volume 
# The UNIv3 subgraph has a field called `gasUsed` but actually it is not the gas used, it represents the limit 
# Usually, trades are settling lower than this limit. Hence it is unfair to UNIv3 prices to adjust with this as is. 
# Moreover, where the data is available to observe on Etherscan, we did not find an open source api call that can 
# deliver this. If you know of any, please share!). 
# Hence, a manual sample was taken for UNIv3 transactions and a generous lower bound is chosen of 60%  
# Moreover another estimator is the ETH price which is chosen to be 1500. This could be improved by fetching the
# ETH price on a block by block basis. 

limit_used_gas_factor = 0.6

uni_df['percent_adjust'] = ((uni_df['gasUSD'] / uni_df['swaps_amountOutUSD'])* limit_used_gas_factor) +1

/tmp/ipykernel_237871/521485859.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['percent_adjust'] = ((uni_df['gasUSD'] / uni_df['swaps_amountOutUSD'])* limit_used_gas_factor) +1


In [54]:
uni_df['uni_in_out_ratio_adjusted'] = uni_df['uni_in_out_ratio'] * uni_df['percent_adjust']

/tmp/ipykernel_237871/3374149081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['uni_in_out_ratio_adjusted'] = uni_df['uni_in_out_ratio'] * uni_df['percent_adjust']


In [104]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD,percent_adjust,uni_in_out_ratio_adjusted
0,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b0d000000,0x6ccb438774e78d276f7ff61ec2c6a10d544d5106483afa5a7f0e7990a4d7f77b,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824178,1678772495,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,2.822205e+23,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.679753e+20,120464.0,None,1.601539e+11,0x60594a405d53811d3bc4766596efd80fd545a270,282466.989679,282220.488073,167.975307,1680.130808,1.601539e-07,28.939167,1.000061,1680.234087
1,0x966bcc3b8e3dd0e13208208b2e95eafddf3c87813006fd36651dd342a4352acc16000000,0x966bcc3b8e3dd0e13208208b2e95eafddf3c87813006fd36651dd342a4352acc,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824149,1678772147,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,1.265266e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,7.520070e+18,120103.0,None,2.556527e+10,0x60594a405d53811d3bc4766596efd80fd545a270,12645.737410,12652.662316,7.520070,1682.519152,2.556527e-08,4.605699,1.000219,1682.886826
2,0x386d24d52ebb99cb41a14bc76459ffcb9d3268ee81cebe8112325ecb5f19fed517000000,0x386d24d52ebb99cb41a14bc76459ffcb9d3268ee81cebe8112325ecb5f19fed5,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824099,1678771547,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,6.180051e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,3.676467e+19,153329.0,None,2.177077e+10,0x60594a405d53811d3bc4766596efd80fd545a270,61930.434502,61800.508369,36.764666,1680.975672,2.177077e-08,5.007136,1.000049,1681.057217
3,0x3a948990f8d467535e0804c4108913f24d4394d39b5a325d135998f936146afd02000000,0x3a948990f8d467535e0804c4108913f24d4394d39b5a325d135998f936146afd,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824091,1678771451,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,8.504112e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,5.059155e+19,109912.0,None,3.209627e+11,0x60594a405d53811d3bc4766596efd80fd545a270,85221.960542,85041.116105,50.591553,1680.935069,3.209627e-07,52.916485,1.000373,1681.561310
4,0xa13aca138e3cf246d46a449ab28acd2206ebf878459e68fe58e90de3f70b1edb72000000,0xa13aca138e3cf246d46a449ab28acd2206ebf878459e68fe58e90de3f70b1edb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16824042,1678770863,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,4.022544e+22,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2.365715e+19,1000000.0,None,1.166374e+11,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,40125.204944,40225.441874,23.657149,1700.350380,1.166374e-07,174.956143,1.002616,1704.798758


In [56]:
uni_df.to_csv('uni_df.csv')

### 4. Merge COW and  Univ3 

Notes: 

- In cow, the sell and buy are from user's point of view. In Uni, the in and out are from pool point of view. 
so sell = in and buy = out 

- since ratios are defined as sell / buy then we can compare if cow< uni it means that cow price was better from the user's point of view 

In [57]:
# merge trades and swaps on timestamp value, tokens, and direction of trade. We use outer join because we want to keep all trades and swaps data and backfill swap values
cow_uni_df = pd.merge(cow_df, uni_df, left_on=['trades_timestamp','trades_sellToken_id','trades_buyToken_id'], right_on=['swaps_timestamp','swaps_tokenIn_id','swaps_tokenOut_id'])

In [58]:
# re sorting by desceding timesamps keeps things tidy
cow_uni_df.sort_values(by='trades_timestamp', ascending=False, inplace=True)

In [59]:
cow_uni_df.shape

(56214, 45)

In [60]:
cow_uni_df.dtypes

Unnamed: 0                     int64
trades_id                     object
trades_timestamp               int64
trades_gasPrice                int64
trades_feeAmount              object
trades_txHash                 object
trades_settlement_id          object
trades_sellAmount             object
sell_token_decimal             int64
trades_buyAmount              object
buy_token_decimal              int64
trades_sellToken_id           object
trades_buyToken_id            object
trades_order_id               object
sell_token_symbol             object
buy_token_symbol              object
trades_sellAmountUsd         float64
sell_amount                  float64
buy_amount                   float64
cow_sell_buy_ratio           float64
swaps_id                      object
swaps_hash                    object
swaps_protocol_id             object
swaps_blockNumber              int64
swaps_timestamp                int64
swaps_tokenIn_id              object
tokenIn_symbol                object
t

In [61]:
cow_uni_df.head(5)

,Unnamed: 0,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,trades_sellAmountUsd,sell_amount,buy_amount,cow_sell_buy_ratio,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD,percent_adjust,uni_in_out_ratio_adjusted
0,0,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b|0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af|126,1679120339,15532267543,3165845034591732,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,8000000000000000000,18,4172524128209352386385,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b,WETH,AURA,14511.550918,8.000000,4172.524128,0.001917,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29aff8000000,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852814,1679120339,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.307835e+18,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,AURA,18,6.839601e+20,463680.0,None,1.553227e+10,0x4be410e2ff6a5f1718ada572afa9e8d26537242b,2369.202625,1.307835,683.960136,0.001912,1.553227e-08,10.803003,1.002736,0.001917
44,1010,0x2455d24f03b8f9565472f50341e4eca61ea8033496c4928f2c9723c11963ae5cb874dd4d665dc22e7744ed38da743aee16dfd0d964156a67|0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831|299,1679119223,15670056908,2902502739473372,0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831,0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831,166735803561240633,18,297693697,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x2455d24f03b8f9565472f50341e4eca61ea8033496c4928f2c9723c11963ae5cb874dd4d665dc22e7744ed38da743aee16dfd0d964156a67,WETH,USDC,302.436576,0.166736,297.693697,0.000560,0x7657d46ed70f798c5448e923005773db122b3f5e5d99736a63f1582c443e07455e000000,0x7657d46ed70f798c5448e923005773db122b3f5e5d99736a63f1582c443e0745,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852721,1679119223,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,6.953636e+17,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1.263533e+09,571812.0,None,1.785000e+10,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1263.533203,0.695364,1263.533203,0.000550,1.785000e-08,15.310266,1.007270,0.000554
43,1010,0x2455d24f03b8f9565472f50341e4eca61ea8033496c4928f2c9723c11963ae5cb874dd4d665dc22e7744ed38da743aee16dfd0d964156a67|0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831|299,1679119223,15670056908,2902502739473372,0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831,0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831,166735803561240633,18,297693697,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x2455d24f03b8f9565472f50341e4eca61ea8033496c4928f2c9723c11963ae5cb874dd4d665dc22e7744ed38da743aee16dfd0d964156a67,WETH,USDC,302.436576,0.166736,297.693697,0.000560,0xa96b0afdf9dc5b442f35dd6ea669da0c02f46863644b10200fe50a8c6ea57a7b0e000000,0xa96b0afdf9dc5b442f35dd6ea669da0c02f46863644b10200fe50a8c6ea57a7b,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852721,1679119223,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.368582e+18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2.486786e+09,652965.0

###  5. Group trades in same block and derive blockMax and VWAP  

In [62]:
# Validate grouping 
# Check that transactions are grouped by timestamp, sell token, and buy token. i.e. all transactions have the same 
# three are grouped together. 

cow_uni_df_group_test = cow_uni_df.groupby(['trades_timestamp', 'trades_sellToken_id', 'trades_buyToken_id'])

In [63]:
group_sizes = cow_uni_df_group_test.size()
print(group_sizes)

trades_timestamp  trades_sellToken_id                         trades_buyToken_id                        
1646095203        0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xd5d86fc8d5c0ea1ac1ac5dfab6e529c9967a45e9    1
1646095700        0x6bea7cfef803d1e3d5f7c0103f7ded065644e197  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
                  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xf4d2888d29d722226fafa5d9b24f9164c092421e    2
1646096769        0x3472a5a71965499acd81997a54bba8d852c6e53d  0x2260fac5e5542a773aa44fbcfedf7c193bc2c599    1
1646099346        0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
                                                                                                           ..
1679115707        0xc0c293ce456ff0ed870add98a0828dd4d2903dbf  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
1679116835        0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8    1
1679118815     

In [64]:
# Split the merged cow_uni_df into just the univ3 side to allow further manipulation and grouping by block. 
uni_temp_df = cow_uni_df.loc[:, 'swaps_id':]

In [65]:
uni_temp_df.shape

(56214, 25)

In [66]:
uni_temp_df.dtypes

swaps_id                      object
swaps_hash                    object
swaps_protocol_id             object
swaps_blockNumber              int64
swaps_timestamp                int64
swaps_tokenIn_id              object
tokenIn_symbol                object
tokenIn_decimal                int64
swaps_amountIn               float64
swaps_tokenOut_id             object
tokenOut_symbol               object
tokenOut_decimal               int64
swaps_amountOut              float64
swaps_gasLimit               float64
swaps_gasUsed                 object
swaps_gasPrice               float64
swaps_pool_id                 object
swaps_amountOutUSD           float64
univ3_In_amount              float64
univ3_Out_amount             float64
uni_in_out_ratio             float64
gas_wei                      float64
gasUSD                       float64
percent_adjust               float64
uni_in_out_ratio_adjusted    float64
dtype: object

In [67]:
uni_temp_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD,percent_adjust,uni_in_out_ratio_adjusted
0,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29aff8000000,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852814,1679120339,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.307835e+18,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,AURA,18,6.839601e+20,463680.0,None,1.553227e+10,0x4be410e2ff6a5f1718ada572afa9e8d26537242b,2369.202625,1.307835,683.960136,0.001912,1.553227e-08,10.803003,1.002736,0.001917
44,0x7657d46ed70f798c5448e923005773db122b3f5e5d99736a63f1582c443e07455e000000,0x7657d46ed70f798c5448e923005773db122b3f5e5d99736a63f1582c443e0745,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852721,1679119223,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,6.953636e+17,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1.263533e+09,571812.0,None,1.785000e+10,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1263.533203,0.695364,1263.533203,0.000550,1.785000e-08,15.310266,1.007270,0.000554
43,0xa96b0afdf9dc5b442f35dd6ea669da0c02f46863644b10200fe50a8c6ea57a7b0e000000,0xa96b0afdf9dc5b442f35dd6ea669da0c02f46863644b10200fe50a8c6ea57a7b,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852721,1679119223,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.368582e+18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2.486786e+09,652965.0,None,1.568809e+10,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,2486.786496,1.368582,2486.786496,0.000550,1.568809e-08,15.365661,1.003707,0.000552
45,0x56bf3d5b76b917f513737a59eb02d48ab30ed49f1ffd0db55c9c36abc496b6857b010000,0x56bf3d5b76b917f513737a59eb02d48ab30ed49f1ffd0db55c9c36abc496b685,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852688,1679118815,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,3.000000e+17,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,5.449544e+08,424148.0,None,1.554291e+10,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,544.954413,0.300000,544.954413,0.000551,1.554291e-08,9.888740,1.010888,0.000556
56187,0x751f0abaab13bd2a9a9ef86c5b592262fc3149290edc800263aa3e25c7b1ac3ab7000000,0x751f0abaab13bd2a9a9ef86c5b592262fc3149290edc800263aa3e25c7b1ac3a,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852525,1679116835,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1.876510e+09,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,GPT,18,1.677379e+22,722590.0,None,1.544552e+10,0xdb02d6827220475ece91893255fdf57bed51aee3,1868.835826,1876.509737,16773.794534,0.111872,1.544552e-08,16.741167,1.005375,0.112473


In [68]:
# Validate grouping 

grouped_df_test = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id'])
group_sizes = grouped_df_test.size()
print(group_sizes)

swaps_timestamp  swaps_tokenIn_id                            swaps_tokenOut_id                         
1646095203       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xd5d86fc8d5c0ea1ac1ac5dfab6e529c9967a45e9    1
1646095700       0x6bea7cfef803d1e3d5f7c0103f7ded065644e197  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
                 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xf4d2888d29d722226fafa5d9b24f9164c092421e    2
1646096769       0x3472a5a71965499acd81997a54bba8d852c6e53d  0x2260fac5e5542a773aa44fbcfedf7c193bc2c599    1
1646099346       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
                                                                                                          ..
1679115707       0xc0c293ce456ff0ed870add98a0828dd4d2903dbf  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
1679116835       0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8    1
1679118815       0xc02aa

In [69]:
# BEST price in block 

In [70]:
# Group and perform MIN operator to obtain highest value in univ3. (i.e. The maximum price in the block) 
grouped_df_min = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id']).agg({'uni_in_out_ratio_adjusted': 'min'})

In [71]:
grouped_df_min.shape

(38021, 1)

In [72]:
grouped_df_min.sort_values(by='swaps_timestamp', ascending=False, inplace=True)

In [73]:
grouped_df_min = grouped_df_min.rename(columns={'uni_in_out_ratio_adjusted': 'univ3BlockMin'})

In [74]:
grouped_df_min

,,,univ3BlockMin
swaps_timestamp,swaps_tokenIn_id,swaps_tokenOut_id,
1679120339,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0.001917
1679119223,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.000552
1679118815,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.000556
1679116835,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,0.112473
1679115707,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,540.832570
...,...,...,...
1646099346,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.000364
1646096769,0x3472a5a71965499acd81997a54bba8d852c6e53d,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,5147.660987
1646095700,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xf4d2888d29d722226fafa5d9b24f9164c092421e,0.000431


**VWAP of block**

The calculation is a simple weighted average formula. The weights used are the volume of the trade. Since trades in a group and have the same buy token and same sell token, we chose the `univ3_Out_amount` as our weights. Using the `univ3_In_amount` should have the same result. 

VWAP of blocks. 

In [75]:
uni_temp_df2 = uni_temp_df

In [76]:
uni_temp_df2['weighted_ratio'] = uni_temp_df2['univ3_Out_amount'] * uni_temp_df2['uni_in_out_ratio_adjusted']

In [77]:
uni_temp_df2.head(2)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD,percent_adjust,uni_in_out_ratio_adjusted,weighted_ratio
0,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29aff8000000,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852814,1679120339,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,1.307835e+18,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,AURA,18,6.839601e+20,463680.0,None,1.553227e+10,0x4be410e2ff6a5f1718ada572afa9e8d26537242b,2369.202625,1.307835,683.960136,0.001912,1.553227e-08,10.803003,1.002736,0.001917,1.311413
44,0x7657d46ed70f798c5448e923005773db122b3f5e5d99736a63f1582c443e07455e000000,0x7657d46ed70f798c5448e923005773db122b3f5e5d99736a63f1582c443e0745,0x1f98431c8ad98523631ae4a59f267346ea31f984,16852721,1679119223,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,6.953636e+17,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1.263533e+09,571812.0,None,1.785000e+10,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1263.533203,0.695364,1263.533203,0.000550,1.785000e-08,15.310266,1.007270,0.000554,0.700419


In [78]:
# chose weights as swap amounts out as they include all data in integers. 
grouped_df_vwap = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id']).agg({'univ3_Out_amount':'sum','weighted_ratio':'sum'})

In [79]:
grouped_df_vwap['univ3_vwap'] = grouped_df_vwap['weighted_ratio'] / grouped_df_vwap['univ3_Out_amount'] 

In [80]:
grouped_df_vwap.sort_values(by='swaps_timestamp', ascending=False, inplace=True)

In [81]:
grouped_df_vwap.shape

(38021, 3)

In [82]:
grouped_df_vwap

,,,univ3_Out_amount,weighted_ratio,univ3_vwap
swaps_timestamp,swaps_tokenIn_id,swaps_tokenOut_id,,,
1679120339,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,683.960136,1.311413,0.001917
1679119223,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3750.319699,2.074075,0.000553
1679118815,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,544.954413,0.303266,0.000556
1679116835,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,16773.794534,1886.595684,0.112473
1679115707,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.207079,111.995250,540.832570
...,...,...,...,...,...
1646099346,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,292.363124,0.106443,0.000364
1646096769,0x3472a5a71965499acd81997a54bba8d852c6e53d,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0.019736,101.595833,5147.660987
1646095700,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xf4d2888d29d722226fafa5d9b24f9164c092421e,3680.947782,1.626786,0.000442


#### Step 3: Merge cow trades with univ3_blockMin and univ3_vwap 

In [83]:
# merge trades and swaps on timestamp value, tokens, and direction of trade. We use outer join because we want to keep all trades and swaps data and backfill swap values
cow_univ3_merge1_df = pd.merge(cow_df, grouped_df_min, left_on=['trades_timestamp','trades_sellToken_id','trades_buyToken_id'], right_on=['swaps_timestamp','swaps_tokenIn_id','swaps_tokenOut_id'])

In [84]:
cow_univ3_merge1_df.sort_values(by='trades_timestamp', ascending=False, inplace=True)

In [85]:
cow_univ3_merge1_df.shape

(38474, 21)

In [86]:
cow_univ3_merge1_df

,Unnamed: 0,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,trades_sellAmountUsd,sell_amount,buy_amount,cow_sell_buy_ratio,univ3BlockMin
0,0,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b|0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af|126,1679120339,15532267543,3165845034591732,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,8000000000000000000,18,4172524128209352386385,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b,WETH,AURA,14511.550918,8.000000,4172.524128,0.001917,0.001917
40,1010,0x2455d24f03b8f9565472f50341e4eca61ea8033496c4928f2c9723c11963ae5cb874dd4d665dc22e7744ed38da743aee16dfd0d964156a67|0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831|299,1679119223,15670056908,2902502739473372,0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831,0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831,166735803561240633,18,297693697,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x2455d24f03b8f9565472f50341e4eca61ea8033496c4928f2c9723c11963ae5cb874dd4d665dc22e7744ed38da743aee16dfd0d964156a67,WETH,USDC,302.436576,0.166736,297.693697,0.000560,0.000552
41,1011,0xeccec80c358e4fbfc12ca964bbb8f898e3e92dbdaf32bfad1d74dc19fed6ba4490a69b1a180f60c0059f149577919c778ce2b9e1641553ec|0x1081a33ec928c2e87b8cf14d352738d4031283d0a9e9665c3f85d916feca1fba|20,1679118815,15598939314,2479263910813908,0x1081a33ec928c2e87b8cf14d352738d4031283d0a9e9665c3f85d916feca1fba,0x1081a33ec928c2e87b8cf14d352738d4031283d0a9e9665c3f85d916feca1fba,16173933943922295871,18,29402532404,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeccec80c358e4fbfc12ca964bbb8f898e3e92dbdaf32bfad1d74dc19fed6ba4490a69b1a180f60c0059f149577919c778ce2b9e1641553ec,WETH,USDC,29327.834310,16.173934,29402.532404,0.000550,0.000556
38447,266948,0x3328bffaa0f9ccb1cdf90992800dac4cdb1ee1be8604c57ec26566892b999e6f244291d3f74380d7cd10b861622aff1216fc2697641550ff|0x751f0abaab13bd2a9a9ef86c5b592262fc3149290edc800263aa3e25c7b1ac3a|90,1679116835,15445519957,5876855,0x751f0abaab13bd2a9a9ef86c5b592262fc3149290edc800263aa3e25c7b1ac3a,0x751f0abaab13bd2a9a9ef86c5b592262fc3149290edc800263aa3e25c7b1ac3a,1882386592,6,16773794533923756332497,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,0x3328bffaa0f9ccb1cdf90992800dac4cdb1ee1be8604c57ec26566892b999e6f244291d3f74380d7cd10b861622aff1216fc2697641550ff,USDC,GPT,1882.386592,1882.386592,16773.794534,0.112222,0.112473
36059,253199,0xd9fa87a4c0e6afd91df677d22d24287abe8d45b655c5ada31f1a4adfd9a7b553bfe334b816404e21577cef84b1d3cf703563751264154c87|0x8bfabd3bfed11b39e85cb45202132135a1b500d211b59d1b4a249413a382720b|99,1679115707,15046265934,1595952574076320256,0x8bfabd3bfed11b39e85cb45202132135a1b500d211b59d1b4a249413a382720b,0x8bfabd3bfed11b39e85cb45202132135a1b500d211b59d1b4a249413a382720b,111342027933865887996,18,207079336476055710,18,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xd9fa87a4c0e6afd91df677d22d24287abe8d45b655c5ada31f1a4adfd9a7b553bfe334b816404e21577cef84b1d3cf703563751264154c87,AURA,WETH,383.513418,111.342028,0.207079,537.678118,540.832570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8750,13210,0xfa6fb3f3e7ff657c646612f128599a6adb3927ae1146663e2fc1b70630f0268a56feea4525a1fe9a8ebedfd0429016d1e7cecf03621d822b|0xc2f6220f8d7c1e5457ea267c82101c819a6ea525fd199322164ae8944c017301|7,164609934

In [87]:
# Perform the merge to include vwap result
cow_univ3_merge2_df = pd.merge(cow_univ3_merge1_df, grouped_df_vwap, 
                               left_on=['trades_timestamp', 'trades_sellToken_id', 'trades_buyToken_id'], 
                               right_on=['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id'], 
                               how='left')


In [88]:
cow_univ3_merge2_df.head(5)

,Unnamed: 0,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,trades_sellAmountUsd,sell_amount,buy_amount,cow_sell_buy_ratio,univ3BlockMin,univ3_Out_amount,weighted_ratio,univ3_vwap
0,0,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b|0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af|126,1679120339,15532267543,3165845034591732,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,0x36f9701a90752838fadeabd2cbd1eabdd082ae93999aae59d4a58f79f8fd29af,8000000000000000000,18,4172524128209352386385,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0x1199d15dc132c42b4903410899acab17832a44d3a5364c19d081034c635a2a42277de71b4f1aeeec027b75ccc0761b7d9d5a54c164155e9b,WETH,AURA,14511.550918,8.000000,4172.524128,0.001917,0.001917,683.960136,1.311413,0.001917
1,1010,0x2455d24f03b8f9565472f50341e4eca61ea8033496c4928f2c9723c11963ae5cb874dd4d665dc22e7744ed38da743aee16dfd0d964156a67|0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831|299,1679119223,15670056908,2902502739473372,0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831,0xea140077faf278e69a0bfbfc83908e57031de5f4d26e16ea90953cfeb5d16831,166735803561240633,18,297693697,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x2455d24f03b8f9565472f50341e4eca61ea8033496c4928f2c9723c11963ae5cb874dd4d665dc22e7744ed38da743aee16dfd0d964156a67,WETH,USDC,302.436576,0.166736,297.693697,0.000560,0.000552,3750.319699,2.074075,0.000553
2,1011,0xeccec80c358e4fbfc12ca964bbb8f898e3e92dbdaf32bfad1d74dc19fed6ba4490a69b1a180f60c0059f149577919c778ce2b9e1641553ec|0x1081a33ec928c2e87b8cf14d352738d4031283d0a9e9665c3f85d916feca1fba|20,1679118815,15598939314,2479263910813908,0x1081a33ec928c2e87b8cf14d352738d4031283d0a9e9665c3f85d916feca1fba,0x1081a33ec928c2e87b8cf14d352738d4031283d0a9e9665c3f85d916feca1fba,16173933943922295871,18,29402532404,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeccec80c358e4fbfc12ca964bbb8f898e3e92dbdaf32bfad1d74dc19fed6ba4490a69b1a180f60c0059f149577919c778ce2b9e1641553ec,WETH,USDC,29327.834310,16.173934,29402.532404,0.000550,0.000556,544.954413,0.303266,0.000556
3,266948,0x3328bffaa0f9ccb1cdf90992800dac4cdb1ee1be8604c57ec26566892b999e6f244291d3f74380d7cd10b861622aff1216fc2697641550ff|0x751f0abaab13bd2a9a9ef86c5b592262fc3149290edc800263aa3e25c7b1ac3a|90,1679116835,15445519957,5876855,0x751f0abaab13bd2a9a9ef86c5b592262fc3149290edc800263aa3e25c7b1ac3a,0x751f0abaab13bd2a9a9ef86c5b592262fc3149290edc800263aa3e25c7b1ac3a,1882386592,6,16773794533923756332497,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,0x3328bffaa0f9ccb1cdf90992800dac4cdb1ee1be8604c57ec26566892b999e6f244291d3f74380d7cd10b861622aff1216fc2697641550ff,USDC,GPT,1882.386592,1882.386592,16773.794534,0.112222,0.112473,16773.794534,1886.595684,0.112473
4,253199,0xd9fa87a4c0e6afd91df677d22d24287abe8d45b655c5ada31f1a4adfd9a7b553bfe334b816404e21577cef84b1d3cf703563751264154c87|0x8bfabd3bfed11b39e85cb45202132135a1b500d211b59d1b4a249413a382720b|99,1679115707,15046265934,1595952574076320256,0x8bfabd3bfed11b39e85cb45202132135a1b500d211b59d1b4a249413a382720b,0x8bfabd3bfed11b39e85cb45202132135a1b500d211b59d1b4a249413a382720b,111342027933865887996,18,207079336476055710,18,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xd9fa87a4c0e6afd91df677d22d24287abe8d45b655c5ada31f1a4adfd9a7b553bfe334b816404e21577cef84b1d3cf703563751264154c87,AURA,WETH,383.513418,111.342028,0.207079,537.678118,540.832570,0.207079,111.995250,540.832570


In [89]:
cow_univ3_merge2_df.shape

(38474, 24)

In [90]:
cow_univ3_merge2_df.dtypes

Unnamed: 0                int64
trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         object
trades_txHash            object
trades_settlement_id     object
trades_sellAmount        object
sell_token_decimal        int64
trades_buyAmount         object
buy_token_decimal         int64
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
sell_token_symbol        object
buy_token_symbol         object
trades_sellAmountUsd    float64
sell_amount             float64
buy_amount              float64
cow_sell_buy_ratio      float64
univ3BlockMin           float64
univ3_Out_amount        float64
weighted_ratio          float64
univ3_vwap              float64
dtype: object

In [91]:
unique_cow_txns = len(cow_univ3_merge2_df['trades_id'].unique())
unique_cow_txns

38474

In [92]:
cow_univ3_merge2_df.to_csv('zero_arb_cow_univ3_raw_data.csv')

### 6. Analysis 

Look at:
how many times cow beat maxBlock pricing. 
How many times cow beat vwap pricing 
What is sd of differences with vwap? 
For the times COW beat pricing, what tokens? 
by How much? 
group by token directions and see how stats differ. 
by How much? 
differences stats


Since we have defined ratios to be sell/ price then a higher number = worse pricing. 

In [93]:
prices_df = cow_univ3_merge2_df[['trades_sellAmountUsd','cow_sell_buy_ratio', 'univ3BlockMin', 'univ3_vwap']]

In [94]:
prices_df

,trades_sellAmountUsd,cow_sell_buy_ratio,univ3BlockMin,univ3_vwap
0,14511.550918,0.001917,0.001917,0.001917
1,302.436576,0.000560,0.000552,0.000553
2,29327.834310,0.000550,0.000556,0.000556
3,1882.386592,0.112222,0.112473,0.112473
4,383.513418,537.678118,540.832570,540.832570
...,...,...,...,...
38469,67250.776274,0.000342,0.000364,0.000364
38470,888.341130,5066.803268,5147.660987,5147.660987
38471,5802.934817,4608.721742,4681.355466,4681.355466
38472,1770.139797,0.000445,0.000431,0.000442


In [95]:
# calculate percentage difference 
prices_df['cow_blockMin_diff'] = ((prices_df['cow_sell_buy_ratio'] - prices_df['univ3BlockMin']) / prices_df['univ3BlockMin']) * 100

/tmp/ipykernel_237871/332473052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['cow_blockMin_diff'] = ((prices_df['cow_sell_buy_ratio'] - prices_df['univ3BlockMin']) / prices_df['univ3BlockMin']) * 100


In [96]:
# calculate percentage difference 
prices_df['cow_vwap_diff'] = ((prices_df['cow_sell_buy_ratio'] - prices_df['univ3_vwap']) / prices_df['univ3_vwap']) * 100

/tmp/ipykernel_237871/3309710702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['cow_vwap_diff'] = ((prices_df['cow_sell_buy_ratio'] - prices_df['univ3_vwap']) / prices_df['univ3_vwap']) * 100


In [97]:
prices_df

,trades_sellAmountUsd,cow_sell_buy_ratio,univ3BlockMin,univ3_vwap,cow_blockMin_diff,cow_vwap_diff
0,14511.550918,0.001917,0.001917,0.001917,-0.004047,-0.004047
1,302.436576,0.000560,0.000552,0.000553,1.395761,1.275197
2,29327.834310,0.000550,0.000556,0.000556,-1.152207,-1.152207
3,1882.386592,0.112222,0.112473,0.112473,-0.223105,-0.223105
4,383.513418,537.678118,540.832570,540.832570,-0.583259,-0.583259
...,...,...,...,...,...,...
38469,67250.776274,0.000342,0.000364,0.000364,-6.108443,-6.108443
38470,888.341130,5066.803268,5147.660987,5147.660987,-1.570766,-1.570766
38471,5802.934817,4608.721742,4681.355466,4681.355466,-1.551553,-1.551553
38472,1770.139797,0.000445,0.000431,0.000442,3.372496,0.731096


In [107]:
# Total volume USD of all trades
cow_total_volume_usd = prices_df['trades_sellAmountUsd'].sum()
cow_total_volume_usd

4805589503.277952

In [98]:
# Comparison with blockMax 

In [99]:
# Count the number of values less than 0
blockMin_lt_zero = (prices_df['cow_blockMin_diff'] < 0).sum()

# Count the number of values greater than 0
blockMin_gt_zero = (prices_df['cow_blockMin_diff'] > 0).sum()

# Count the number of values equal to 0
blockMin_eq_zero = (prices_df['cow_blockMin_diff'] == 0).sum()


total_count = len(prices_df)
blockMin_gt_zero_pct = blockMin_gt_zero / total_count * 100
blockMin_eq_zero_pct = blockMin_eq_zero/ total_count * 100
blockMin_lt_zero_pct = blockMin_lt_zero/ total_count * 100



print("Number of values < 0:", blockMin_lt_zero, '/', blockMin_lt_zero_pct, '%')
print("Number of values > 0:", blockMin_gt_zero, '/', blockMin_gt_zero_pct, '%')
print("Number of values = 0:", blockMin_eq_zero, '/', blockMin_eq_zero_pct, '%')

Number of values < 0: 23913 / 62.15366221344284 %
Number of values > 0: 14561 / 37.84633778655716 %
Number of values = 0: 0 / 0.0 %


In [109]:
blockMin_lt_zero_volume = prices_df.loc[prices_df['cow_blockMin_diff'] < 0, 'trades_sellAmountUsd'].sum()
blockMin_gt_zero_volume = prices_df.loc[prices_df['cow_blockMin_diff'] > 0, 'trades_sellAmountUsd'].sum()
blockMin_eq_zero_volume = prices_df.loc[prices_df['cow_blockMin_diff'] == 0, 'trades_sellAmountUsd'].sum()

print("volume of values < 0 of total volume:", blockMin_lt_zero_volume, '/', blockMin_lt_zero_volume/cow_total_volume_usd*100)
print("volume of values > 0 of total volume:", blockMin_gt_zero_volume, '/', blockMin_gt_zero_volume/cow_total_volume_usd*100)
print("volume of values = 0 of total volume:", blockMin_eq_zero_volume, '/', blockMin_eq_zero_volume/cow_total_volume_usd*100)

volume of values < 0 of total volume: 2692188744.576729 / 56.022028988126294
volume of values > 0 of total volume: 2113400758.701223 / 43.9779710118737
volume of values = 0 of total volume: 0.0 / 0.0


In [100]:
# Compute the average of values greater than 0
blockMin_gt_zero_mean = prices_df.loc[prices_df['cow_blockMin_diff'] > 0, 'cow_blockMin_diff'].mean()

# Compute the average of values less than 0
blockMin_lt_zero_mean = prices_df.loc[prices_df['cow_blockMin_diff'] < 0, 'cow_blockMin_diff'].mean()

print("Average of values > 0:", blockMin_gt_zero_mean)
print("Average of values < 0:", blockMin_lt_zero_mean)



Average of values > 0: 0.5420988532391042
Average of values < 0: -0.5171515562354131


In [101]:
# Comparison with VWAP 

In [102]:
# Count the number of values less than 0
VWAP_lt_zero = (prices_df['cow_vwap_diff'] < 0).sum()

# Count the number of values greater than 0
VWAP_gt_zero = (prices_df['cow_vwap_diff'] > 0).sum()

# Count the number of values equal to 0
VWAP_eq_zero = (prices_df['cow_vwap_diff'] == 0).sum()


total_count = len(prices_df)
VWAP_gt_zero_pct = VWAP_gt_zero / total_count * 100
VWAP_eq_zero_pct = VWAP_eq_zero/ total_count * 100
VWAP_lt_zero_pct = VWAP_lt_zero/ total_count * 100




print("Number of values < 0:", VWAP_lt_zero, '/', VWAP_lt_zero_pct, '%')
print("Number of values > 0:", VWAP_gt_zero, '/', VWAP_gt_zero_pct, '%')
print("Number of values = 0:", VWAP_eq_zero, '/', VWAP_eq_zero_pct, '%')

Number of values < 0: 25480 / 66.22654260019753 %
Number of values > 0: 12994 / 33.77345739980247 %
Number of values = 0: 0 / 0.0 %


In [110]:
VWAP_lt_zero_volume = prices_df.loc[prices_df['cow_vwap_diff'] < 0, 'trades_sellAmountUsd'].sum()
VWAP_gt_zero_volume = prices_df.loc[prices_df['cow_vwap_diff'] > 0, 'trades_sellAmountUsd'].sum()
VWAP_eq_zero_volume = prices_df.loc[prices_df['cow_vwap_diff'] == 0, 'trades_sellAmountUsd'].sum()

print("volume of values < 0 of total volume:", VWAP_lt_zero_volume, '/', VWAP_lt_zero_volume/cow_total_volume_usd*100)
print("volume of values > 0 of total volume:", VWAP_gt_zero_volume, '/', VWAP_gt_zero_volume/cow_total_volume_usd*100)
print("volume of values = 0 of total volume:", VWAP_eq_zero_volume, '/', VWAP_eq_zero_volume/cow_total_volume_usd*100)

volume of values < 0 of total volume: 3060955608.3862 / 63.69573610684567
volume of values > 0 of total volume: 1744633894.8917522 / 36.30426389315433
volume of values = 0 of total volume: 0.0 / 0.0


In [103]:
# Compute the average of values greater than 0
VWAP_gt_zero_mean = prices_df.loc[prices_df['cow_vwap_diff'] > 0, 'cow_vwap_diff'].mean()

# Compute the average of values less than 0
VWAP_lt_zero_mean = prices_df.loc[prices_df['cow_vwap_diff'] < 0, 'cow_vwap_diff'].mean()

print("Average of values > 0:", VWAP_gt_zero_mean)
print("Average of values < 0:", VWAP_lt_zero_mean)

Average of values > 0: 0.5500775776440184
Average of values < 0: -0.5118626937234333
